## 8.1 EM algorithm for binary matrix completion

### (a)

In [1]:
import numpy as np
from tabulate import tabulate

In [2]:
movies = np.loadtxt('hw8_movies.txt', dtype=str)
len(movies)

76

In [3]:
# compute movie to idx dict
movieIdxDict = {movies[i] : i for i in range(len(movies))}
len(movieIdxDict)

76

In [4]:
ids = np.loadtxt('hw8_ids.txt', delimiter='\n', dtype=str)
len(ids)

362

In [5]:
# compute id to idx dict
idIdxDict = {ids[i] : i for i in range(len(ids))}
len(idIdxDict)

362

In [6]:
ratings = np.loadtxt('hw8_ratings.txt', dtype=str)
ratings.shape

(362, 76)

In [7]:
meanPopRatingsPerMovie = []
for movie in movies:
    mi = movieIdxDict[movie]
    recCount = 0
    seenCount = 0
    for si in range(len(ids)):
        if ratings[si,mi] != '?':
            seenCount += 1
            if ratings[si, mi] == '1':
                recCount += 1
    meanPopRatingsPerMovie.append((100 * recCount / seenCount, movie))

In [8]:
meanPopRatingsPerMovie.sort()
print(tabulate(meanPopRatingsPerMovie, headers=['Rec(%)', 'Title'], floatfmt='.5f'))

  Rec(%)  Title
--------  --------------------------------------------
35.89744  I_Feel_Pretty
37.71429  Fifty_Shades_of_Grey
45.65217  Hustlers
47.36842  The_Last_Airbender
51.51515  Magic_Mike
54.05405  Fast_&_Furious:_Hobbs_&_Shaw
55.84416  The_Shape_of_Water
58.40708  Prometheus
58.62069  Phantom_Thread
58.95522  World_War_Z
59.35829  Star_Wars:_The_Force_Awakens
59.67742  Rocketman
60.00000  Chappaquidick
61.90476  Bridemaids
62.42775  Man_of_Steel
62.82051  American_Hustle
63.88889  Terminator:_Dark_Fate
65.15152  Room
65.62500  Good_Boys
65.66265  Pokemon_Detective_Pikachu
65.88235  Fast_Five
67.83217  Mad_Max:_Fury_Road
69.01408  Drive
70.00000  Us
70.68966  The_Help
72.00000  Pitch_Perfect
72.09302  Jurassic_World
72.41379  Frozen
73.60406  X-Men:_First_Class
73.63636  The_Revenant
73.91304  Ex_Machina
74.24242  Avengers:_Age_of_Ultron
75.47170  La_La_Land
75.60976  Midnight_in_Paris
76.36364  Manchester_by_the_Sea
76.85950  Once_Upon_a_Time_in_Hollywood
77.41935  Three_Billbo

### (e)

In [9]:
probR = np.loadtxt('hw8_probR_init.txt')
probR.shape

(76, 4)

In [10]:
probZ = np.loadtxt('hw8_probZ_init.txt')
probZ

array([0.25, 0.25, 0.25, 0.25])

In [11]:
k = 4 # number of movie-goer types
N = len(movies) # number of movies
T = len(ids) # number of students

In [12]:
def estep_num(i, t, pz, pr, ratings):
    j_rec, = np.where(ratings[t,:] == '1')
    j_notrec, = np.where(ratings[t,:] == '0')
    numer = pz[i] * np.prod(pr[j_rec,i]) * np.prod(1-pr[j_notrec,i])
    return numer

def estep_denom(t, pz, priors, ratings):
    denom = 0
    j_rec, = np.where(ratings[t,:] == '1')
    j_notrec, = np.where(ratings[t,:] == '0')
    for i in range(k):
        denom += pz[i]*np.prod(priors[j_rec,i])*np.prod(1-priors[j_notrec,i])
    return denom

def e_step(pz, pr, ratings):
    L = 0
    posteriors = np.empty([k,T], dtype='float32')
    for t in range(T):
        L += np.log(log_likelihood(t, pz, pr, ratings))
        e_denom = estep_denom(t, pz, pr, ratings)
        # e-step - update posteriors
        for i in range(k):
            posteriors[i,t] = estep_num(i, t, pz, pr, ratings) / e_denom
    L /= T
    return L, posteriors

def mstep_prz(i, j, posteriors, priors, ratings):
    # sum over students who recommended movie j (I(r_j,1))
    t_seen, = np.where(ratings[:,j] == '1')
    numer_seen = np.sum(posteriors[i,t_seen])
    # sum over students who have not seen movie j
    t_unseen, = np.where(ratings[:,j] == '?')
    numer_unseen = priors[j,i]*np.sum(posteriors[i,t_unseen])
    return numer_seen+numer_unseen

def m_step(posteriors, pr):
    pz_new = np.empty(4)
    pr_new = np.empty([T, k])
    for i in range(k):
        temp = np.sum(posteriors[i,:])
        pz_new[i] = temp/T
        for j in range(N):
            pr_new[j,i] = mstep_prz(i, j, posteriors, pr, ratings) / temp
    return pz_new, pr_new

def log_likelihood(t, pz, priors, ratings):
    log_li = 0
    for i in range(k):
        j_rec, = np.where(ratings[t,:] == '1')
        j_notrec, = np.where(ratings[t,:] == '0')
        log_li += pz[i]*np.prod(priors[j_rec,i])*np.prod(1-priors[j_notrec,i])
    return log_li

In [13]:
print_itr = 1
posteriors = np.empty([k,T], dtype='float32')
for iteration in range(256+1):
    L, posteriors = e_step(probZ, probR, ratings)
    probZ, probR = m_step(posteriors, probR)
    if iteration == 0 or iteration == print_itr:
        print('iteration: %d \t log-likelihood L = %.4f' % (iteration, L))
        if iteration == print_itr: print_itr *= 2

iteration: 0 	 log-likelihood L = -27.0358
iteration: 1 	 log-likelihood L = -17.5604
iteration: 2 	 log-likelihood L = -16.0024
iteration: 4 	 log-likelihood L = -15.0606
iteration: 8 	 log-likelihood L = -14.5016
iteration: 16 	 log-likelihood L = -14.2638
iteration: 32 	 log-likelihood L = -14.1802
iteration: 64 	 log-likelihood L = -14.1701
iteration: 128 	 log-likelihood L = -14.1640
iteration: 256 	 log-likelihood L = -14.1637


### (f)

In [14]:
PID = 'A92408103'
myIdx = idIdxDict[PID]
myRatings = ratings[myIdx,:]
print('My Ratings:')
print(tabulate(np.column_stack((movies, myRatings)), headers=['Movie', 'Rating']))

My Ratings:
Movie                                         Rating
--------------------------------------------  --------
Inception                                     1
The_Social_Network                            1
Black_Swan                                    ?
Shutter_Island                                ?
The_Last_Airbender                            0
Harry_Potter_and_the_Deathly_Hallows:_Part_1  1
Iron_Man_2                                    0
Toy_Story_3                                   1
Fast_Five                                     ?
Thor                                          1
Captain_America:_The_First_Avenger            1
Harry_Potter_and_the_Deathly_Hallows:_Part_2  1
Bridemaids                                    ?
The_Girls_with_the_Dragon_Tattoo              ?
X-Men:_First_Class                            ?
Drive                                         ?
The_Help                                      ?
Midnight_in_Paris                             1
Prometheus      

In [15]:
unseen, = np.where(myRatings == '?')
unseen = movies[unseen]
unseen

array(['Black_Swan', 'Shutter_Island', 'Fast_Five', 'Bridemaids',
       'The_Girls_with_the_Dragon_Tattoo', 'X-Men:_First_Class', 'Drive',
       'The_Help', 'Django_Unchained', 'Pitch_Perfect', '21_Jump_Street',
       'Magic_Mike', 'Now_You_See_Me', '12_Years_a_Slave',
       'American_Hustle', 'Room', 'The_Hateful_Eight', 'The_Revenant',
       'I_Feel_Pretty', 'Chappaquidick', 'Phantom_Thread', 'Rocketman',
       'The_Farewell', 'Good_Boys', 'Us', 'Pokemon_Detective_Pikachu',
       'Hustlers'], dtype='<U44')

In [16]:
posteriors[:,idIdxDict[PID]]

array([ 1.6173378e-04,  9.9983829e-01,  0.0000000e+00, -3.3353604e-08],
      dtype=float32)

In [17]:
unseenRatings = []
for movie in unseen:
    movieIdx = movieIdxDict[movie]
    pr = np.dot(probR[movieIdx,:], posteriors[:, myIdx])
    unseenRatings.append((pr, movie))
unseenRatings.sort()
unseenRatings.reverse()
print(tabulate(unseenRatings, headers=['Expected Rating', 'Unseen Movie']))

  Expected Rating  Unseen Movie
-----------------  --------------------------------
         0.999986  The_Farewell
         0.898581  The_Hateful_Eight
         0.896749  Black_Swan
         0.89264   The_Revenant
         0.889808  Shutter_Island
         0.886514  Django_Unchained
         0.858479  Drive
         0.849043  12_Years_a_Slave
         0.828579  The_Girls_with_the_Dragon_Tattoo
         0.775531  Room
         0.738154  21_Jump_Street
         0.729265  The_Help
         0.695902  Us
         0.6674    I_Feel_Pretty
         0.665514  Bridemaids
         0.653227  Now_You_See_Me
         0.637099  Pitch_Perfect
         0.626004  X-Men:_First_Class
         0.620568  Rocketman
         0.610027  Phantom_Thread
         0.607726  Pokemon_Detective_Pikachu
         0.527355  Magic_Mike
         0.506522  Hustlers
         0.499898  Chappaquidick
         0.444657  Fast_Five
         0.396266  American_Hustle
         0.338861  Good_Boys
